In [1]:
import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from IPython.core.display import display, HTML

pd.set_option('display.max_rows',400)
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
"""
Read in one week of turnstile data to extract station names
Source: Richard Chiou
"""

def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [160903]


In [3]:
"""
source: https://stackoverflow.com/questions/43892459/check-if-geo-point-is-inside-or-outside-of-polygon
"""


from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def isPointinPolygon(lat, long, coord_arr):
    polygon = Polygon(coord_arr)
    point = Point(long,lat)
    return point.within(polygon)


One of the goals of our project was to determine which neighbhorhoods had the greatest decrease in MTA ridership. In order to do that, we needed to assign each station to a neighborhood.

In this notebook, the MTA turnstile data set was normalized to the subway station coordinates data set by station name. This allows for assignment of coordinates to each station in the turnstile data set, followed by assignment of neighborhood to each station.

Station names were normalized by:
   1. First transformation: character regex
   2. Final transformation: re-ordering each string to [0-9][a-z] 
   3. Using fuzzywuzzy for fuzzy string match of final transformations

In [4]:
"""Regex dict for data cleaning"""

replace_values = {' - ': '_',
                  '-': '_',
                  ' ': '_',
                  '/': '_',
                  '--': '_',
                  r"\(": "",
                  r"\)": ""       
                 }             

In [5]:
"""
Character regex for both datasets
"""

station_names = pd.read_csv("http://web.mta.info/developers/data/nyct/subway/Stations.csv")
station_names_stop_names = pd.DataFrame(station_names['Stop Name'].unique(), columns=['original_station_names'])
station_names_stop_names['first_clean'] = station_names_stop_names.original_station_names.str.upper().replace(replace_values, regex=True)

turnstiles_df = get_data(week_nums)
turnstiles_df['StationLine'] = turnstiles_df.STATION + turnstiles_df.LINENAME
turnstile_station_names = pd.DataFrame(turnstiles_df.STATION.unique(),columns=['original_turnstile'])
turnstile_station_names['first_clean'] = turnstile_station_names.original_turnstile.str.upper().replace(replace_values, regex=True)


In [6]:
"""
Re-ordering of station coordinates dataset
"""

"""Splitting cleaned name into delimit by _"""
split_name = station_names_stop_names.first_clean.str.split(pat='_',expand=True)

"""Handling numeric portion of split,cleaned name"""
num_split = pd.DataFrame()
for i in range(0,len(split_name.columns-1)):
    num_split[i] = split_name[i].str.findall(r'(\d+)')
num_split_list = num_split.values.tolist()

for i in range(len(num_split_list)):
    num_split_list[i] = [x for x in num_split_list[i] if x]

num_split_clean = pd.DataFrame(num_split_list)
for i in range(len(num_split_clean.columns)):
    num_split_clean[i] = num_split_clean[i].str[0]
num_split_clean['cleaned_num'] = num_split_clean[num_split_clean.columns[:]].apply(lambda x: '_'.join(x.dropna().astype(str)),axis=1)

"""Handling char portion of split, cleaned name"""
char_split = pd.DataFrame()
for i in range(0,len(split_name.columns-1)):
    char_split[i] = split_name[i].str.findall(r'(\D+)')
char_split_list = char_split.values.tolist()

for i in range(len(char_split_list)):
    char_split_list[i] = [x for x in char_split_list[i] if x!=[]]

char_split_clean = pd.DataFrame(char_split_list)

for i in range(len(num_split_clean.columns)):
    char_split_clean[i] = char_split_clean[i].str[0]

char_split_clean['cleaned_num']=char_split_clean[char_split_clean.columns[:]].apply(lambda x: '_'.join(x.dropna().astype(str)),axis=1)

station_names_stop_names['final_clean_station_names'] = (num_split_clean.cleaned_num + char_split_clean.cleaned_num).str.replace('[','',regex=True).str.replace(']','',regex=True)
cleaned_station_name_data = station_names_stop_names.sort_values(by=['final_clean_station_names']).reset_index()

#station_names_stop_names[station_names_stop_names.duplicated(['final_clean_station_names'])]




In [7]:
"""
Duplication check of transformed string in station names
"""
cleaned_station_name_data[cleaned_station_name_data.duplicated(['final_clean_station_names'])]


,index,original_station_names,first_clean,final_clean_station_names


In [8]:
"""
Re-ordering of MTA turnstiles dataset
"""

"""Splitting cleaned name into delimit by _"""
split_name = turnstile_station_names.first_clean.str.split(pat='_',expand=True)

"""Handling numeric portion of split,cleaned name"""
num_split = pd.DataFrame()
for i in range(0,len(split_name.columns-1)):
    num_split[i] = split_name[i].str.findall(r'(\d+)')
num_split_list = num_split.values.tolist()

for i in range(len(num_split_list)):
    num_split_list[i] = [x for x in num_split_list[i] if x]

num_split_clean = pd.DataFrame(num_split_list)
for i in range(len(num_split_clean.columns)):
    num_split_clean[i] = num_split_clean[i].str[0]
num_split_clean['cleaned_num'] = num_split_clean[num_split_clean.columns[:]].apply(lambda x: '_'.join(x.dropna().astype(str)),axis=1)

"""Handling char portion of split, cleaned name"""
char_split = pd.DataFrame()
for i in range(0,len(split_name.columns-1)):
    char_split[i] = split_name[i].str.findall(r'(\D+)')
char_split_list = char_split.values.tolist()

for i in range(len(char_split_list)):
    char_split_list[i] = [x for x in char_split_list[i] if x!=[]]

char_split_clean = pd.DataFrame(char_split_list)

for i in range(len(num_split_clean.columns)):
    char_split_clean[i] = char_split_clean[i].str[0]

char_split_clean['cleaned_num']=char_split_clean[char_split_clean.columns[:]].apply(lambda x: '_'.join(x.dropna().astype(str)),axis=1)

"""Concat cleaned num and cleaned char"""

turnstile_station_names['final_clean_station_names_turnstiles'] = (num_split_clean.cleaned_num + char_split_clean.cleaned_num).str.replace('[','',regex=True).str.replace(']','',regex=True)
cleaned_turnstile_data = turnstile_station_names.sort_values(by=['final_clean_station_names_turnstiles']).reset_index()



In [12]:
"""
Duplication check of transformed string in MTA turnstile dataset

Can ignore this case, manual check conducted for duplication case

"""
cleaned_turnstile_data[cleaned_turnstile_data.duplicated(['final_clean_station_names_turnstiles'])]

,index,original_turnstile,first_clean,final_clean_station_names_turnstiles
94,221,4 AV-9 ST,4_AV_9_ST,4_9AV_ST


In [10]:
"""
Generate final table of original names with transformed names for both data sets
"""

cleaned_concat = pd.concat([cleaned_station_name_data,cleaned_turnstile_data], axis=1)[['original_station_names','final_clean_station_names', 'original_turnstile','final_clean_station_names_turnstiles']].fillna("")


In [13]:
"""
Use fuzzywuzzy for string match of final transformed names for both data sets
"""

final_match = []

for turnstile_name_clean in list(cleaned_concat.final_clean_station_names_turnstiles):
    station_name_clean = process.extractOne(turnstile_name_clean, list(cleaned_concat.final_clean_station_names),scorer=fuzz.token_sort_ratio)[0]
    turnstile_name_match = cleaned_concat.original_turnstile[cleaned_concat.final_clean_station_names_turnstiles == turnstile_name_clean].values[0]
    station_name_match = cleaned_concat.original_station_names[cleaned_concat.final_clean_station_names == station_name_clean].values[0]
    final_match.append([turnstile_name_match,station_name_match])

final_match_dict = pd.DataFrame(final_match,columns=['Turnstile_Station_Name','Station_Name']).replace(r'^\s*$', np.nan, regex=True).dropna().sort_values(by=['Station_Name']).reset_index(drop=True)


In [14]:
"""
Read in and normalize station coordinate data
"""

station_names.columns = ['Station_ID','Complex_ID','Stop_ID','Division','Line','StopName','Borough','DayTimeRoutes','Structure','Latitude','Longitude','NorthDirectionLabel','SouthDirectionLabel']

station_names['StopName'] = station_names.StopName.astype(str)
station_names['Latitude'] = station_names.Latitude.astype(str)
station_names['Longitude'] = station_names.Longitude.astype(str)


In [15]:
station_names['merged'] = station_names[['StopName','Latitude','Longitude']].apply(lambda x: '_'.join(x), axis = 1) 

In [16]:
station_locations = pd.DataFrame(station_names.merged.unique())[0].str.split(pat='_',expand=True)
station_locations.columns = ['StopName','Longitude','Latitude']

station_locations

,StopName,Longitude,Latitude
0,Astoria - Ditmars Blvd,40.775036,-73.91203399999999
1,Astoria Blvd,40.770258,-73.917843
2,30 Av,40.766779,-73.921479
3,Broadway,40.76182,-73.925508
4,36 Av,40.756803999999995,-73.929575
...,...,...,...
488,Prince's Bay,40.525507,-74.200064
489,Pleasant Plains,40.52241,-74.21784699999999
490,Richmond Valley,40.519631,-74.229141
491,Tottenville,40.512764000000004,-74.251961


In [17]:
"""
Matching coordinates to MTA turnstile station names
"""

merged_station_location = []
for station_name in list(final_match_dict.Station_Name):
    turnstile_name_final = final_match_dict.Turnstile_Station_Name[final_match_dict.Station_Name == station_name].values[0]
    longitude = station_locations.Longitude[station_locations.StopName == station_name].values[0]
    latitude = station_locations.Latitude[station_locations.StopName == station_name].values[0]
    merged_station_location.append([turnstile_name_final,longitude,latitude])
    
final_merged_station_location = pd.DataFrame(merged_station_location,columns=['TurnstileData_StationName','Longitude','Latitude'])
final_merged_station_location.to_csv("station_locations_dict.csv",index=False)


In [18]:
"""
Regex dict for normalizing station coordinate data set
"""

replace_values_neighborhoods = {r"\(": "",
                                r"\)": "",
                                'MULTIPOLYGON ': '',
                               ', ': ',',
                               ' ':','}


In [19]:
"""
Regex normalization of neighborhoods data set
"""

neighborhoods_df = pd.read_csv('nynta.csv')[['BoroName','NTAName','the_geom']]
neighborhoods_df.columns = ['Borough','NTA','geom']
neighborhoods_df = neighborhoods_df.replace(replace_values_neighborhoods,regex=True)

In [20]:
"""
Reformat coordinate array from neighborhoods data set for polygon mapping
"""

coord_array_list = []
for coords in list(neighborhoods_df.geom):
    split = np.char.split(coords, sep=',').tolist()
    split_list = [[i] for i in split]
    coord_array = np.array([j+i for i,j in zip(split_list[::2], split_list[1::2])])
    coord_array_list.append(coord_array)
    
neighborhoods_df['geom_array'] = pd.Series(coord_array_list)

In [21]:
neighborhoods_df.to_csv("NTA_locations_polygons.csv",index=False)

In [22]:
"""
For every station, assign station to nieghborhood
"""

final_assign = []

for station_row in final_merged_station_location.itertuples():
    for array in neighborhoods_df.geom_array:
        if isPointinPolygon(float(station_row.Latitude), float(station_row.Longitude), array):
            nta = neighborhoods_df.NTA[neighborhoods_df.geom_array.astype(str) == str(array)].values[0]
            borough = neighborhoods_df.Borough[neighborhoods_df.geom_array.astype(str)==str(array)].values[0]
            final_assign.append([station_row.TurnstileData_StationName,nta,borough])


final = pd.DataFrame(final_assign)

In [24]:
"""
Manual match of remaining strings
"""
final.columns=['Station','NTA','Borough']
success_stations = pd.Series(final.Station.unique())
need_to_match_stations = final_merged_station_location.TurnstileData_StationName[final_merged_station_location.TurnstileData_StationName.isin(success_stations)==False].unique()


In [26]:
"""
For strings with more than 1 fuzzywuzzy best match, rank matches by occurance and pick match with best rank
"""

NTA_counts = pd.DataFrame(final.NTA.value_counts()).reset_index()
NTA_counts.columns = ['NTA','counts']
counts_list = []
for rows in final.itertuples():
    count = NTA_counts.counts[NTA_counts.NTA == rows.NTA]
    counts_list.append(count.values[0])
    
counts_series = pd.Series(counts_list)

final['counts_rank'] = counts_series
station_neighborhood_assignments = final.groupby(['Station'])[['NTA','counts_rank']].max().reset_index().drop(['counts_rank'],axis=1)

In [27]:
"""
Completion of matchlist
"""

manual_match = pd.DataFrame({'Station':['4AV-9 ST','EASTN PKWY-MUSM','RIT-ROOSEVELT'], 'NTA':['Park Slope-Gowanus','Prospect Heights','Lenox Hill-Roosevelt Island']})
station_neighborhood_assignments = station_neighborhood_assignments.append(manual_match).reset_index()
station_neighborhood_assignments.NTA = station_neighborhood_assignments.NTA.str.upper().replace(',',' ',regex=True)
station_neighborhood_assignments = station_neighborhood_assignments[['Station','NTA']]
station_neighborhood_assignments.to_csv('station_neighborhood_assignments.csv',index=False)
